In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

Welcome to JupyROOT 6.22/02


In [21]:
name='gold_47_SC_90m_211220'
name.split('_')

angle=int(name.split('_')[1])*(1 if name.split('_')[2]=='DC' else -1)

file=open('gold_47_SC_90m_211220/Stat_gold_47_SC_90m_211220.txt')
line=file.readlines()
for l in line:
    if (l.startswith("Readout_time")):
        time=float(l.split('=')[1].strip()[:-1])
    if (l.startswith("Strobe")):
        Strobe=int(l.split('=')[1].strip()[:-1])
    if (l.startswith("Gap")):
        Gap=int(l.split('=')[1].strip()[:-1])
        
weight=3600*(Gap+Strobe)/(Strobe*time)

os.system("python Packet_analyzer.py -f gold_47_SC_90m_211220 -p -A DB -d 1 10 -b 1000 -s %i -w %f " % (angle,weight))



0

1 Packet files found
Parallel processing with DBSCAN
Completeness : 0.0%

Estimated noise points= 262
Estimated clusters= 1072
Estimated mean area= 5.412313432835821
Process time =  0.4160325527191162
The current working directory is /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer
Directory /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer/Analyzed_Data/gold_47_SC_90m_211220 alredy present


# Analysis on a specific dataset

In [4]:
files=os.listdir()
for name in files:
    if name.startswith("gold_"):
        angle=int(name.split('_')[1])*(1 if name.split('_')[2]=='DC' else -1)

        file=open(name+"/Stat_"+name+".txt")
        line=file.readlines()
        for l in line:
            if (l.startswith("Readout_time")):
                time=float(l.split('=')[1].strip()[:-1])
            if (l.startswith("Strobe")):
                Strobe=int(l.split('=')[1].strip()[:-1])
            if (l.startswith("Gap")):
                Gap=int(l.split('=')[1].strip()[:-1])
                
        #weight that gives count/hour
        weight=3600*(Gap+Strobe)/(Strobe*time)

        os.system("python Packet_analyzer.py -f " + name + " -p -A DB -d 1 10 -b 1000 -s %i -w %f " % (angle,weight))

1 Packet files found
Parallel processing with DBSCAN
Completeness : 0.0%

Estimated noise points= 443
Estimated clusters= 1967
Estimated mean area= 5.383833248601932
Process time =  0.6666514873504639
The current working directory is /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer
Directory /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer/Analyzed_Data/gold_59_SC_240m_201220 alredy present
1 Packet files found
Parallel processing with DBSCAN
Completeness : 0.0%

Estimated noise points= 178
Estimated clusters= 761
Estimated mean area= 5.390275952693824
Process time =  0.3382422924041748
The current working directory is /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer
Directory /home/gabriele/GitHub/Rutherford/RutherfordScattering_AdvLab/simulations/ALPIDE/Packet_analyzer/Analyzed_Data/gold_53_SC_90m_211220 alredy present
1 Packet files found
Paral

In [5]:
! $ROOTSYS/bin/hadd Tot_63_47.root Analyzed_Data/gold_*/*.root

hadd Target file: Tot_63_47.root
hadd compression setting for all output: 1
hadd Source file 1: Analyzed_Data/gold_47_SC_90m_211220/gold_47_SC_90m_211220.root
hadd Source file 2: Analyzed_Data/gold_49_SC_90m_211220/gold_49_SC_90m_211220.root
hadd Source file 3: Analyzed_Data/gold_51_SC_90m_211220/gold_51_SC_90m_211220.root
hadd Source file 4: Analyzed_Data/gold_53_SC_90m_211220/gold_53_SC_90m_211220.root
hadd Source file 5: Analyzed_Data/gold_59_SC_240m_201220/gold_59_SC_240m_201220.root
hadd Source file 6: Analyzed_Data/gold_61_SC_240m_201220/gold_61_SC_240m_201220.root
hadd Source file 7: Analyzed_Data/gold_63_SC_90m_201220/gold_63_SC_90m_201220.root
hadd Target path: Tot_63_47.root:/


In [6]:
ROOT.enableJSVis()

root_file_1=ROOT.TFile.Open("Tot_63_47.root")
Data_tree = root_file_1.Get("tree")
Data_tree.Print()
Areas=Data_tree.AsMatrix(['Rareas'])
Means=Data_tree.AsMatrix(['Rmeanx','Rmeany'])
Theta=Data_tree.AsMatrix(['Rtheta'])
Weight=Data_tree.AsMatrix(['Rweight'])
Ratios=Data_tree.AsMatrix(['Rratios'])

******************************************************************************
*Tree    :tree      : file                                                   *
*Entries :     8157 : Total =          202152 bytes  File  Size =      95548 *
*        :          : Tree compression factor =   2.10                       *
******************************************************************************
*Br    0 :Rareas    : Rareas/F                                               *
*Entries :     8157 : Total  Size=      33628 bytes  File Size  =       9777 *
*Baskets :        7 : Basket Size=      32000 bytes  Compression=   3.39     *
*............................................................................*
*Br    1 :Rmeanx    : Rmeanx/F                                               *
*Entries :     8157 : Total  Size=      33628 bytes  File Size  =      23874 *
*Baskets :        7 : Basket Size=      32000 bytes  Compression=   1.39     *
*...................................................

In [7]:
hm = ROOT.TH2D('hm','Hitmap',128,0,1024,64,0,512)

for coordinate in Means:
    hm.Fill(coordinate[0],coordinate[1])
    
    
HC=ROOT.TCanvas()
hm.Draw('COLZ')
HC.Draw()



In [8]:
ar_max=int(np.amax(Areas))
nbins=int((ar_max-1))

arHist = ROOT.TH1F('arHist', 'Area',nbins,1,ar_max)

for value in Areas:
    arHist.Fill(value)

AC=ROOT.TCanvas()
arHist.Draw()
AC.Draw()

r_max=int(np.amax(Ratios))
nbins=50

RHist = ROOT.TH1F('RHist', 'Ratios',nbins,0.5,r_max+0.5)

for value in Ratios:
    RHist.Fill(value)

RC=ROOT.TCanvas()
RHist.Draw()
RC.Draw()

In [18]:
Area_thr=13
Ratio_thr=1.51

A_disc_index=np.where(Areas<Area_thr)
Areas_disc=np.delete(Areas,A_disc_index)
Means_disc=np.delete(Means,A_disc_index,axis=0)
Theta_disc=np.delete(Theta,A_disc_index)
Weight_disc=np.delete(Weight,A_disc_index)
Ratios_disc=np.delete(Ratios,A_disc_index)

R_disc_index=np.where(Ratios_disc>Ratio_thr)
Areas_disc=np.delete(Areas_disc,R_disc_index)
Means_disc=np.delete(Means_disc,R_disc_index,axis=0)
Theta_disc=np.delete(Theta_disc,R_disc_index)
Weight_disc=np.delete(Weight_disc,R_disc_index)
Ratios_disc=np.delete(Ratios_disc,R_disc_index)

In [19]:
th_max=(np.amax(Theta_disc))
th_min=(np.amin(Theta_disc))
nbins=50

ThHist = ROOT.TH1F('ThHist', 'Theta weighted',nbins,th_min,th_max)

for value,weight in zip(Theta_disc,Weight_disc):
    ThHist.Fill(value,weight)

    
TC=ROOT.TCanvas()
ThHist.Draw('HIST')
TC.Draw()   

Warning in <TFile::Append>: Replacing existing TH1: ThHist (Potential memory leak).


In [20]:
ar_max=int(np.amax(Areas_disc))
print(ar_max)
nbins=int((ar_max))

ArHist = ROOT.TH1F('ArHist', 'Area',nbins,1,ar_max+1)

for value in Areas_disc:
    ArHist.Fill(value)

AC=ROOT.TCanvas()
ArHist.Draw()
AC.Draw()

r_max=np.amax(Ratios_disc)
print(r_max)
nbins=30

RHist = ROOT.TH1F('RHist', 'Ratios',nbins,0.5,r_max+0.5)

for value in Ratios_disc:
    RHist.Fill(value)

RC=ROOT.TCanvas()
RHist.Draw()
RC.Draw()



#MxHist = ROOT.TH1F('MxHist', 'X Means',16,0,1023)
#
#for value in Means_disc[:,0]:
#    MxHist.Fill(value)
#
#MxC=ROOT.TCanvas()
#MxHist.Draw()
#MxC.Draw()

31
1.5


Warning in <TFile::Append>: Replacing existing TH1: ArHist (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: RHist (Potential memory leak).
